In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, date
import cv2
import time
from PIL import Image
import os
import matplotlib as mpl
mpl.rc('font',family='Arial Narrow')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.cbook import get_sample_data
from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage,
                                  AnnotationBbox)
from scipy.io.wavfile import write
import sys

Type in the course and date for the video below, using the following format as an example:

- course = Psych100
- course2 = PSYCH 100
- theDate = yyyy-mm-dd
- term = F2019

`course` and `theDate` should be exactly as they appear in the video title. `course2` is usually `course` with all letters capitalized and a space added between letters and numbers.

In [ ]:
#select date of video and course
course = 'Psych261'
course2 = 'PSYCH 261'
theDate = '2019-10-30'
term = 'F2019'

a dictionary of start times for all classes included

In [ ]:
startDict = dict(zip(['PSYCH 261'], 
                     ['13:00:00.000000']))

In [ ]:
## TO DO retrieve threshold from metadata, after programming it in
def read_meta(course, date):
    with open('./../output/{}/{}_meta.txt'.format(course, date)) as metadata:
        meta = metadata.read() 
    dur, fps, fin, frames = meta.split(' ')
    return int(dur), np.float32(fps), int(fin), int(frames)

def im_to_plot(imfile, axis, x, y, yoffset=0, scale=0.5):
    arr_img = plt.imread(imfile, format='gif')

    imagebox = OffsetImage(arr_img, zoom=scale)
    imagebox.image.axes = axis
    im = AnnotationBbox(imagebox, (x, y),
                        xybox=(x, y+yoffset),
                        xycoords='data',
                        boxcoords='data',
                        pad=0,
                        arrowprops=dict(
                            arrowstyle="->",
                            connectionstyle="angle,angleA=0,angleB=90,rad=3")
                        )
    axis.add_artist(im)
    
def im_to_vert(imfile, axis, x, y, xoffset=0, scale=0.5):
    arr_img = plt.imread(imfile, format='gif')

    imagebox = OffsetImage(arr_img, zoom=scale)
    imagebox.image.axes = axis
    im = AnnotationBbox(imagebox, (x, y),
                        xybox=(x+xoffset, y),
                        xycoords='data',
                        boxcoords='data',
                        pad=0,
                        arrowprops=dict(width=10, headwidth=14, headlength=30)
                        
                        )
    axis.add_artist(im)

In [ ]:
# Set threshold - to be deprecated after this is coded in the metadata
thresh = 0.95
changes = np.load('./../output/{}/{}_pixelchange.npy'.format(course, theDate))
dur, fps, fin, frames = read_meta(course, theDate)
framespan = np.array(range(len(changes)))
timespan = (framespan * 100)/fps/60
whereCaps = np.zeros((len(framespan)))
whereCaps[changes > (1-thresh)] = 1
whereCaps[changes <= (1-thresh)] = np.nan
numCatches = np.nansum(whereCaps)
Files = [i for i in os.listdir('./../output/{}'.format(course)) if i.split('.')[1]=='gif']
Files = [i for i in Files if i.split('_')[0]==theDate]
Files = sorted(Files, key = lambda x: int(x.split('_')[1]))
onlySwaps = timespan[whereCaps == 1]

assert len(Files) == int(numCatches) == onlySwaps.shape[0]



In [ ]:
### Preprocess probe data

probeDat = pd.read_csv('./../data/{}/Pooled_Data.csv'.format(term))
allTimes = [i.split(' ')[0] for i in list(probeDat['start_time'])]
allDates = list(np.unique(allTimes))


thisCourse = probeDat[probeDat['class'] == course2]

elapsed = []
for row in range(thisCourse.shape[0]): 
    probeTime = datetime.combine(date.min, datetime.strptime(thisCourse['start_time'].iloc[row], '%Y-%m-%d %H:%M:%S.%f').time())
    classStart = startDict[thisCourse['class'].iloc[row]]
    classStart = datetime.combine(date.min, datetime.strptime(classStart, '%H:%M:%S.%f').time())
    difference = probeTime - classStart
    whenProbe = difference.total_seconds()/60
    elapsed.append(whenProbe)

thisCourse['elapsed'] = np.array(elapsed)


onlyResps = thisCourse[thisCourse['response_value'] != 'declined']
onlyResps = onlyResps[onlyResps['response_value'] != 'timeout']
onlyResps['response_value'] = onlyResps['response_value'].astype('float32')

todaysResps = onlyResps[onlyResps['start_time'].str.contains(theDate)]

todaysEngage = todaysResps[todaysResps['probe_type']=='range-engage']
todaysUnder = todaysResps[todaysResps['probe_type']=='range-understand']
courseEngage = onlyResps[onlyResps['probe_type']=='range-engage']
courseUnder = onlyResps[onlyResps['probe_type']=='range-understand']

print('{} total probes found in this class, {} on this day'.format(thisCourse.shape[0], todaysResps.shape[0]))

fig, ax = plt.subplots(2,1, figsize=(6,2))
ax[0].scatter(np.array(todaysEngage['elapsed']), np.array(todaysEngage['response_value']), color='blue')
ax[1].scatter(np.array(todaysUnder['elapsed']), np.array(todaysUnder['response_value']), color='orange')
plt.show()

In [ ]:
ws=20
stdev=2
sp = int((ws/2)-1)

## simplify all points plot for display
newDFs = []
for measure in [courseEngage, courseUnder]:
    measure = measure.sort_values(by=['elapsed'])
    measure['rolling'] = measure['response_value'].rolling(ws, 5, center=True, 
                                                           win_type='gaussian').mean(std=stdev)
    newDFs.append(measure)

courseEngage, courseUnder = newDFs

In [ ]:
ws=6
stdev=2
sp = int((ws/2)-1)

## simplify all points plot for display
newDFs = []
for measure in [todaysEngage, todaysUnder]:
    measure = measure.sort_values(by=['elapsed'])
    measure['rolling'] = measure['response_value'].rolling(ws, 5, center=True, 
                                                           win_type='gaussian').mean(std=stdev)
    newDFs.append(measure)

todaysEngage, todaysUnder = newDFs

In [ ]:
## SELCT A SUBSET OF SLIDES FOR INSTRUCTORS TO RATE
subset = Files[1:-1]

subset=np.array(subset)
print(subset.shape)
np.random.seed(1)
to_test = np.random.choice(subset, size=12, replace=False)
print(to_test)

In [ ]:
# read in video start and stop indices
videodoc = np.load('./../output/{}/{}_videos.npy'.format(course, theDate))
print('detected {} video playing'.format(videodoc.shape[0]))

In [ ]:
## should line be overall term, or just today?
today = False

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(32,16))
trunChanges = np.array(changes)
trunChanges[trunChanges > 0.2] = 0.2
ax.plot(timespan,trunChanges,alpha=0.2)
ax.axhline(1-thresh, c='g', ls='dashed', alpha=0.2, lw=2)
ax.set_yticks(np.arange(0,1.5,0.2))
ax.set_yticklabels(['LO', 'HI', '','','','','',''], fontsize=40, **{'fontname':'Arial Narrow'})
ax.set_xticks(np.arange(0,81,20))
ax.set_xticklabels(np.arange(0,81,20), fontsize=40, **{'fontname':'Arial Narrow'})
ax.set_xlabel('Minutes Elapsed', fontsize=60, **{'fontname':'Arial Narrow'})
ax.set_ylim(0,1.0)
offset = 0.9

for num in range(len(Files)):
    for j in range(videodoc.shape[0]):
        if num not in np.arange(videodoc[j,0]+1,videodoc[j,1]):
            im_to_plot('./../output/{}/{}'.format(course, Files[num]), 
               ax, onlySwaps[num], 1-thresh, yoffset=offset, scale=0.25)
            if np.around(offset,1) == 0.3:
                climb = True
            elif np.around(offset,1) == 0.9:
                climb = False
            if climb:
                offset += 0.1
            else:
                offset -= 0.1
    
        
engage = ax.scatter(np.array(todaysEngage['elapsed']), np.array(todaysEngage['response_value'])/500, 
                    color='blue', s=200, alpha=0.5)
under = ax.scatter(np.array(todaysUnder['elapsed']), np.array(todaysUnder['response_value'])/500, 
                   color='orange', s=200, alpha=0.5)
if today:
    engagestem = 'Engagement Moving Avg'
    understem = 'Understanding Moving Avg'
    allEngage = ax.plot(todaysEngage['elapsed'].iloc[sp:-sp], todaysEngage['rolling'].iloc[sp:-sp]/500, color='blue')
    allUnder = ax.plot(todaysUnder['elapsed'].iloc[sp:-sp], todaysUnder['rolling'].iloc[sp:-sp]/500, color='orange')
else:
    engagestem = 'Term Engagement'
    understem = 'Term Understanding'
    allEngage = ax.plot(courseEngage['elapsed'].iloc[sp:-sp], courseEngage['rolling'].iloc[sp:-sp]/500, color='blue')
    allUnder = ax.plot(courseUnder['elapsed'].iloc[sp:-sp], courseUnder['rolling'].iloc[sp:-sp]/500, color='orange')

allClass = plt.plot()
ax.legend((engage, allEngage[0], under, allUnder[0]), 
          ('Engagement', engagestem, 'Understanding', understem), loc=(0,0.22), 
          mode='expand', ncol=4, fontsize=30)
plt.savefig('./../output/{}/{}_{}.png'.format(course, theDate, course), dpi=300)
plt.show()